<div style="
    background-color: #1e1e1e; 
    color: #e0e0e0; 
    font-family: 'Consolas', 'Monaco', 'Courier New', monospace; 
    padding: 25px; 
    border-radius: 8px; 
    border: 1px solid #333; 
    box-shadow: 0 4px 15px rgba(0,0,0,0.5);
    max-width: 950px;">

<h1 style="
        color: #4fc3f7; 
        border-bottom: 2px solid #0288d1; 
        padding-bottom: 10px; 
        margin-top: 0;
        font-family: 'Segoe UI', sans-serif;
        letter-spacing: 1px;">
        &lt;Optimization: Direct Hashing & Pruning (DHP) /&gt;
    </h1>

 <p style="font-size: 1.05em; color: #b0bec5;">
        The standard Apriori algorithm faces a critical bottleneck during the generation of 
        <strong style="color: #fff;">2-itemsets (C₂)</strong>. DHP addresses this by aggressively pruning candidate pairs early using a hash-based filtering technique.
    </p>

<div style="
        background-color: #263238; 
        border-left: 5px solid #ff5252; 
        padding: 15px; 
        margin: 25px 0; 
        border-radius: 4px;">
        <h3 style="margin-top: 0; color: #ff8a80; font-family: 'Segoe UI', sans-serif;">
            ⚠️ The Bottleneck: Combinatorial Explosion
        </h3>
        <p style="margin-bottom: 0; font-size: 0.95em;">
            If you have 1,000 frequent items in <code>L₁</code>, standard Apriori must generate 
            <code style="color: #ff5252; background-color: rgba(255,255,255,0.1); padding: 2px 6px; border-radius: 4px;">C(1000, 2) ≈ 500,000</code> 
            candidate pairs. Checking all these against the database is slow and memory-intensive.
        </p>
    </div>

<h2 style="color: #81d4fa; font-family: 'Segoe UI', sans-serif;">// How DHP Works</h2>
    <p>
        DHP <strong>"hacks" the first pass</strong>. While counting single items (k=1), it simultaneously gathers data about pairs.
    </p>

<div style="margin-left: 10px;">
        <div style="margin-bottom: 15px;">
            <strong style="color: #fff; font-size: 1.1em;">1. The Hash Function</strong><br>
            During the initial scan, every pair of items in a transaction is passed through a function:
            <div style="
                background-color: #000; 
                color: #a5d6a7; 
                padding: 10px; 
                border-radius: 4px; 
                margin: 10px 0; 
                border-left: 3px solid #66bb6a;">
                bucket_index = ( (order(x) * 10) + order(y) ) % N
            </div>
        </div>

 <div style="margin-bottom: 15px;">
            <strong style="color: #fff; font-size: 1.1em;">2. The Bucket Count</strong><br>
            A Hash Table tracks counts. If a pair maps to bucket #5, we increment bucket #5. We don't store <em>which</em> pair it was, just the frequency.
        </div>

<div>
            <strong style="color: #fff; font-size: 1.1em;">3. The Golden Rule (Pruning)</strong><br>
            After the scan, we check the buckets. <br>
            <span style="color: #ffcc80;">If a bucket's count < min_support, ALL pairs mapping to that bucket are discarded immediately.</span>
        </div>
    </div>

<div style="
        background-color: #1b5e20; 
        background: linear-gradient(145deg, #1b5e20 0%, #2e7d32 100%);
        padding: 20px; 
        margin: 30px 0; 
        border-radius: 6px; 
        color: #e8f5e9;">
        <h3 style="margin-top: 0; color: #fff; font-family: 'Segoe UI', sans-serif;">💡 Concrete Example</h3>
        <p style="margin: 5px 0;"><strong>Scenario:</strong> <code>min_support = 10</code></p>
        <ul style="list-style-type: square; padding-left: 20px;">
            <li>We analyze pair <code>{Milk, Bread}</code>.</li>
            <li>Hash function maps it to <strong>Bucket #5</strong>.</li>
            <li>Total count in Bucket #5 is found to be <strong>8</strong>.</li>
        </ul>
        <hr style="border: 0; border-top: 1px solid rgba(255,255,255,0.3); margin: 10px 0;">
        <p style="margin-bottom: 0; font-weight: bold;">
            Conclusion: Even if all 8 hits were {Milk, Bread}, 8 < 10. This pair is impossible. DROP IT.
        </p>
    </div>

<h2 style="color: #81d4fa; font-family: 'Segoe UI', sans-serif;">// Performance Comparison</h2>
    
 <table style="width: 100%; border-collapse: collapse; margin-top: 15px; font-size: 0.95em;">
        <thead>
            <tr style="border-bottom: 2px solid #4fc3f7;">
                <th style="text-align: left; padding: 12px; color: #4fc3f7;">Feature</th>
                <th style="text-align: left; padding: 12px; color: #b0bec5;">Standard Apriori</th>
                <th style="text-align: left; padding: 12px; color: #fff;">Apriori + DHP</th>
            </tr>
        </thead>
        <tbody>
            <tr style="border-bottom: 1px solid #424242;">
                <td style="padding: 12px; color: #e0e0e0;"><strong>Candidate Gen (C₂)</strong></td>
                <td style="padding: 12px; color: #9e9e9e;">Blindly joins all L₁ items ($\approx n^2/2$).</td>
                <td style="padding: 12px; color: #81c784;">Only generates pairs from frequent buckets.</td>
            </tr>
            <tr style="border-bottom: 1px solid #424242;">
                <td style="padding: 12px; color: #e0e0e0;"><strong>DB Size</strong></td>
                <td style="padding: 12px; color: #9e9e9e;">Full scan every time.</td>
                <td style="padding: 12px; color: #81c784;">Can "trim" DB by removing useless transactions.</td>
            </tr>
        </tbody>
    </table>

</div>


In [1]:
import pandas as pd
from collections import defaultdict
from itertools import combinations
import tracemalloc
import time



In [2]:

# The Raw Data
raw_data = [
    ['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
    ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
    ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
    ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
    ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs'] # Note duplicates
]

# Convert to list of sets (removes duplicate 'Onion' in last row)
dataset = [set(transaction) for transaction in raw_data]

# 2. Parameters
MIN_SUPPORT = 2
BUCKET_COUNT = 7  

# Create a Mapping (String -> Integer) for the Hash Function
# We sort all unique items alphabetically to ensure deterministic IDs
unique_items = sorted(set(item for sublist in dataset for item in sublist))
item_to_id = {item: i+1 for i, item in enumerate(unique_items)}

# Define the Hash Function
def get_hash_bucket(item1, item2, bucket_count=BUCKET_COUNT):
    
    # Get IDs
    id1 = item_to_id[item1]
    id2 = item_to_id[item2]
    
    # Ensure order to make hash commutative: hash(A, B) == hash(B, A)
    first = min(id1, id2)
    second = max(id1, id2)
    
    # Calculate Hash
    val = (first * 10) + second
    return val % bucket_count

# --- DISPLAY SETUP ---
print(f"--- CONFIGURATION ---")
print(f"Transaction Count: {len(dataset)}")
print(f"Min Support: {MIN_SUPPORT}")
print(f"Hash Buckets: {BUCKET_COUNT}")
print("-" * 30)
print("Item ID Mapping (for Hashing):")
for item, idx in item_to_id.items():
    print(f"  {item}: {idx}")


--- CONFIGURATION ---
Transaction Count: 5
Min Support: 2
Hash Buckets: 7
------------------------------
Item ID Mapping (for Hashing):
  Apple: 1
  Corn: 2
  Dill: 3
  Eggs: 4
  Ice cream: 5
  Kidney Beans: 6
  Milk: 7
  Nutmeg: 8
  Onion: 9
  Unicorn: 10
  Yogurt: 11


In [3]:


# Initialization
C1_counts = defaultdict(int)
bucket_counts = defaultdict(int) # This acts as our Hash Table

print("STARTING SCAN 1")

for t_idx, transaction in enumerate(dataset):
    # though sets are fine for combinations
    items = list(transaction)
    
    for item in items:
        C1_counts[item] += 1
        
    #  DHP Special: Hash all 2-item subsets
    # Generate all pairs (combinations of size 2)
    pairs = list(combinations(items, 2))
    
    for pair in pairs:
        bucket_idx = get_hash_bucket(pair[0], pair[1])
        bucket_counts[bucket_idx] += 1
        
    # Optional: Print detail for first transaction only to avoid clutter
    if t_idx == 0:
        print(f"Transaction 1 Pairs processed: {len(pairs)}")
        print(f"Example Pair from T1: {pairs[0]} -> Bucket {get_hash_bucket(pairs[0][0], pairs[0][1])}")

print("\n--- SCAN COMPLETE ---")

# --- FILTERING L1 (Standard Part) ---
# Filter items that meet MIN_SUPPORT
L1 = {item: count for item, count in C1_counts.items() if count >= MIN_SUPPORT}
sorted_L1 = sorted(L1.items(), key=lambda x: x[1], reverse=True)

print(f"\n[L1 Result] Frequent 1-Itemsets (Count >= {MIN_SUPPORT}):")
print(sorted_L1)

# DHP BUCKET RESULTS: nothing pruned here. 
print(f"\n[DHP Result] Hash Table Status (Bucket Counts):")
print(f"{'Bucket ID':<10} | {'Count':<10} | {'Status (Keep/Prune?)'}")
print("-" * 45)

valid_buckets = [] # To store buckets that passed the test
for i in range(BUCKET_COUNT):
    count = bucket_counts[i]
    is_valid = count >= MIN_SUPPORT
    status = " KEEP" if is_valid else "PRUNE"
    if is_valid:
        valid_buckets.append(i)
        
    print(f"{i:<10} | {count:<10} | {status}")

print(f"\nValid Buckets for Next Step: {valid_buckets}")


STARTING SCAN 1
Transaction 1 Pairs processed: 15
Example Pair from T1: ('Nutmeg', 'Kidney Beans') -> Bucket 5

--- SCAN COMPLETE ---

[L1 Result] Frequent 1-Itemsets (Count >= 2):
[('Kidney Beans', 5), ('Eggs', 4), ('Onion', 3), ('Milk', 3), ('Yogurt', 3), ('Nutmeg', 2), ('Corn', 2)]

[DHP Result] Hash Table Status (Bucket Counts):
Bucket ID  | Count      | Status (Keep/Prune?)
---------------------------------------------
0          | 8          |  KEEP
1          | 6          |  KEEP
2          | 5          |  KEEP
3          | 9          |  KEEP
4          | 11         |  KEEP
5          | 8          |  KEEP
6          | 9          |  KEEP

Valid Buckets for Next Step: [0, 1, 2, 3, 4, 5, 6]


Every bucket get keeped because of: B= 7. It's critical to save this.

In [4]:
# L1 keys (items)
l1_items = [x[0] for x in sorted_L1] 

print(f"--- GENERATING C2 CANDIDATES ---")
print(f"Items in L1: {len(l1_items)}")
print(f"Valid Buckets: {valid_buckets}")

C2_candidates = []
rejected_count = 0

# Generate all pairs from L1 items
possible_pairs = list(combinations(l1_items, 2))

print(f"Total possible pairs from L1: {len(possible_pairs)}")
print("-" * 40)

for pair in possible_pairs:
    item1, item2 = pair
    
    # 1. Calculate Hash for this candidate
    bucket = get_hash_bucket(item1, item2)
    
    # 2. DHP CHECK: Is this bucket in our valid list?
    if bucket in valid_buckets:
        C2_candidates.append(frozenset(pair))
        # purely for display:
        # print(f"  Accepted: {pair} (Bucket {bucket})") 
    else:
        rejected_count += 1
        print(f"PRUNED by DHP: {pair} (Bucket {bucket})")

print("-" * 40)
print(f"Candidates Generated (C2 Size): {len(C2_candidates)}")
print(f"Candidates Pruned by DHP: {rejected_count}")


--- GENERATING C2 CANDIDATES ---
Items in L1: 7
Valid Buckets: [0, 1, 2, 3, 4, 5, 6]
Total possible pairs from L1: 21
----------------------------------------
----------------------------------------
Candidates Generated (C2 Size): 21
Candidates Pruned by DHP: 0


In [5]:
def load_dataset():
    return [
        frozenset(['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt']),
        frozenset(['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt']),
        frozenset(['Milk', 'Apple', 'Kidney Beans', 'Eggs']),
        frozenset(['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt']),
        frozenset(['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs'])
    ]

def get_item_id(item_name):
    """Mapping items to integers for hashing."""
    mapping = {
        'Apple': 1, 'Corn': 2, 'Dill': 3, 'Eggs': 4, 
        'Ice cream': 5, 'Kidney Beans': 6, 'Milk': 7, 
        'Nutmeg': 8, 'Onion': 9, 'Unicorn': 10, 'Yogurt': 11
    }
    return mapping.get(item_name, 0)

def get_hash_bucket(item1, item2, bucket_count=7):
    id1 = get_item_id(item1)
    id2 = get_item_id(item2)
    # Ensuring order doesn't change hash (commutative)
    if id1 > id2: id1, id2 = id2, id1
    return (id1 * 10 + id2) % bucket_count

# --- 2. Core Functions ---

def apriori_gen(Lk_minus_1, k):
    """
    Generates Ck from L(k-1).
    For k=2, we use simple combinations.
    For k>2, we join sets that share k-2 items.
    """
    candidates = []
    len_Lk = len(Lk_minus_1)
    lk_list = list(Lk_minus_1)
    
    if k == 2:
        # Simple pairs
        return list(combinations(lk_list, 2))
    
    # Standard Apriori Join for k > 2
    for i in range(len_Lk):
        for j in range(i + 1, len_Lk):
            L1 = list(lk_list[i])
            L2 = list(lk_list[j])
            L1.sort(); L2.sort()
            
            # If first k-2 elements are equal, join them
            if L1[:k-2] == L2[:k-2]:
                candidates.append(lk_list[i] | lk_list[j])
                
    return candidates

def scan_and_count(dataset, candidates):
    """Counts support for a list of candidates."""
    counts = defaultdict(int)
    for tid in dataset:
        for cand in candidates:
            # cand can be a tuple (from combinations) or frozenset
            cand_set = frozenset(cand)
            if cand_set.issubset(tid):
                counts[cand_set] += 1
    return counts

# --- 3. Main Algorithm with Metrics ---

def run_apriori_dhp(min_support=2):
    # Start Metrics
    tracemalloc.start()
    start_time = time.time()
    
    dataset = load_dataset()
    bucket_count = 7
    
    global_frequent_itemsets = {} # Final Result Storage
    
    print("--- Step 1: Init Pass (Count L1 + Hash C2) ---")
    c1_counts = defaultdict(int)
    bucket_counts = defaultdict(int)
    
    for transaction in dataset:
        # Count Items
        for item in transaction:
            c1_counts[item] += 1
            
        # Hash Pairs (DHP)
        items = list(transaction)
        for i in range(len(items)):
            for j in range(i + 1, len(items)):
                b_idx = get_hash_bucket(items[i], items[j], bucket_count)
                bucket_counts[b_idx] += 1

    # Generate L1
    L1 = [item for item, count in c1_counts.items() if count >= min_support]
    L1.sort() # Sorting helps deterministic behavior
    
    # Store L1
    for item in L1:
        global_frequent_itemsets[frozenset([item])] = c1_counts[item]

    print(f"L1 found: {len(L1)} items")
    
    # --- Step 2: Generate C2 using DHP Filter ---
    print("--- Step 2: Generate C2 (with DHP Pruning) ---")
    valid_buckets = {b for b, cnt in bucket_counts.items() if cnt >= min_support}
    
    C2_candidates = []
    l1_pairs = list(combinations(L1, 2))
    
    for pair in l1_pairs:
        b_idx = get_hash_bucket(pair[0], pair[1], bucket_count)
        if b_idx in valid_buckets:
            C2_candidates.append(frozenset(pair))
    
    print(f"C2 Candidates after DHP: {len(C2_candidates)} (out of {len(l1_pairs)} possible)")
    
    # Loop Variables
    current_C = C2_candidates
    k = 2
    
    # --- Step 3: Iterative Loop (L2, L3...) ---
    while len(current_C) > 0:
        print(f"--- Scanning DB for k={k} ---")
        
        # Count Support
        candidates_counts = scan_and_count(dataset, current_C)
        
        # Filter L_k
        L_k = []
        for cand, count in candidates_counts.items():
            if count >= min_support:
                L_k.append(cand)
                global_frequent_itemsets[cand] = count
        
        print(f"L{k} found: {len(L_k)} itemsets")
        
        if len(L_k) <= 1:
            break # Cannot generate C_(k+1) from 1 or 0 items
            
        # Generate Next Candidates (C_k+1) - Standard Join
        k += 1
        current_C = apriori_gen(L_k, k)
        
    # End Metrics
    end_time = time.time()
    current_mem, peak_mem = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    execution_time = (end_time - start_time) * 1000 # ms
    peak_mem_mb = peak_mem / (1024 * 1024)
    
    return global_frequent_itemsets, execution_time, peak_mem_mb

# --- 4. Execution ---

final_results, exec_time, peak_memory = run_apriori_dhp(min_support=2)

print("\n" + "="*50)
print("FINAL OUTPUT")
print("="*50)
print(f"Total Frequent Itemsets Found: {len(final_results)}")
print(f"Execution Time: {exec_time:.4f} ms")
print(f"Peak Memory Usage: {peak_memory:.6f} MB")
print("-" * 50)
print("{Itemset: Support}")
for itemset, support in final_results.items():
    # formatting frozenset to look cleaner
    clean_set = set(itemset) 
    print(f"{clean_set}: {support}")


--- Step 1: Init Pass (Count L1 + Hash C2) ---
L1 found: 7 items
--- Step 2: Generate C2 (with DHP Pruning) ---
C2 Candidates after DHP: 21 (out of 21 possible)
--- Scanning DB for k=2 ---
L2 found: 14 itemsets
--- Scanning DB for k=3 ---
L3 found: 12 itemsets
--- Scanning DB for k=4 ---
L4 found: 5 itemsets
--- Scanning DB for k=5 ---
L5 found: 1 itemsets

FINAL OUTPUT
Total Frequent Itemsets Found: 39
Execution Time: 3.2921 ms
Peak Memory Usage: 0.025233 MB
--------------------------------------------------
{Itemset: Support}
{'Corn'}: 2
{'Eggs'}: 4
{'Kidney Beans'}: 5
{'Milk'}: 3
{'Nutmeg'}: 2
{'Onion'}: 3
{'Yogurt'}: 3
{'Eggs', 'Kidney Beans'}: 4
{'Milk', 'Eggs'}: 2
{'Nutmeg', 'Eggs'}: 2
{'Onion', 'Eggs'}: 3
{'Eggs', 'Yogurt'}: 2
{'Milk', 'Kidney Beans'}: 3
{'Nutmeg', 'Kidney Beans'}: 2
{'Onion', 'Kidney Beans'}: 3
{'Yogurt', 'Kidney Beans'}: 3
{'Milk', 'Yogurt'}: 2
{'Onion', 'Nutmeg'}: 2
{'Nutmeg', 'Yogurt'}: 2
{'Onion', 'Yogurt'}: 2
{'Corn', 'Kidney Beans'}: 2
{'Milk', 'Eggs', 'K